# Road Following - Build TensorRT model

*Original Source: https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/notebooks/road_following/live_demo_build_trt.ipynb*

In this notebook we'll use the model we trained prior in ``road_following_train_resnet18`` to optimize it using TensorRT.

External libraries required:
+ https://github.com/NVIDIA-AI-IOT/torch2trt/tree/v0.3.0

> Important note: you need to run this notebook on a host where the repository torch2trt is installed. We chose to run this script on the Jetbot.
> This means, that we had to upload the prior generated model to the Jetbot.

## Load the trained model

We'll assume that you've already trained the ``best_steering_model_xy.pth`` model and placed it in the right folder.

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')

## TensorRT

> If your setup does not have `torch2trt` installed, you need to first install `torch2trt` by executing the following in the console.
```bash
cd $HOME
git clone https://github.com/NVIDIA-AI-IOT/torch2trt
cd torch2trt
git checkout v0.3.0
sudo python3 setup.py install
```

> **IMPORTANT NOTE** The original section from the official NVIDIA jetbot repo checked out the main branch,
> but that wouldn't install for the current Jetbot version found on https://jetbot.org/master/software_setup/sd_card.html
> (Jetpack 4.5.1-b17 | Jetbot Version 0.4.3 | jetbot-043_nano-4gb-jp45.zip)

Convert and optimize the model using torch2trt for faster inference with TensorRT. Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete.

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

> Naming Convention for completed models: best_steering_model_\[YYYY]\-\[MM]-\[DD]\_\[daily-iteration-step]_trt.pth

In [ ]:
torch.save(model_trt.state_dict(), 'best_steering_model_trt.pth')